<h1 align="center">First of all -- Checking Questions</h1> 

**Вопрос 1**: Можно ли использовать сверточные сети для классификации текстов? Если нет обоснуйте :D, если да то как? как решить проблему с произвольной длинной входа?

<Ответ> Да, можно использовать посимвольный подход.

**Вопрос 2**: Чем LSTM лучше/хуже чем обычная RNN?

<Ответ> при применении LSTM градиент не затухает(или делает это в пределах разумного), а также не взрывается, он сохраняется гораздо дольше. Но он требует больше памяти и вычислений.

**Вопрос 3**:  Выпишите производную $\frac{d c_{n+1}}{d c_{k}}$ для LSTM http://colah.github.io/posts/2015-08-Understanding-LSTMs/, объясните формулу, когда производная затухает, когда взрывается?

<Ответ> https://github.com/g-n-ev/Machine-learning-course/tree/master/hw7/ans_3.jpg

**Вопрос 4**: Зачем нужен TBPTT почему BPTT плох?

<Ответ> BPTT проделывает больше операций, следовательно, требуется больше ресурсов. TBPTT работает быстее, можно применять для претрейнинга.


**Вопрос 5**: Как комбинировать рекуррентные и сверточные сети, а главное зачем? Приведите несколько примеров реальных задач.

<Ответ> Можно разбивать данные путем свертки, а затем обучать их с помощью рекуррентной сети. Это добавит скорости обаботки одного элемента, но может в целом потребовать больше ресурсов

**Вопрос 6**: Объясните интуицию выбора размера эмбединг слоя? почему это опасное место?

<Ответ> порядка процента от исходного словаря. Неаккуратный выбор размера может повлечь потерю большой части данных

* Arseniy Ashuha, you can text me ```ars.ashuha@gmail.com```, Александр Панин

<h1 align="center"> Image Captioning </h1> 

In this seminar you'll be going through the image captioning pipeline. It can help u https://ars-ashuha.ru/slides/2016.11.11_ImageCaptioning/image_captionong.pdf 

To begin with, let us download the dataset of image features from a pre-trained GoogleNet.

In [1]:
!wget https://www.dropbox.com/s/3hj16b0fj6yw7cc/data.tar.gz?dl=1 -O data.tar.gz
!tar -xvzf data.tar.gz

"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
"tar" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


### Data preprocessing

In [1]:
%%time
# Read Dataset
import numpy as np
import pickle

img_codes = np.load("data/image_codes.npy")
captions = pickle.load(open('data/caption_tokens.pcl', 'rb'))

Wall time: 3.8 s


In [2]:
print "each image code is a 1000-unit vector:", img_codes.shape
print img_codes[0,:10]
print '\n\n'
print "for each image there are 5-7 descriptions, e.g.:\n"
print '\n'.join(captions[0])

each image code is a 1000-unit vector: (123287L, 1000L)
[ 1.38901556 -3.82951474 -1.94360816 -0.5317238  -0.03120959 -2.87483215
 -2.9554503   0.6960277  -0.68551242 -0.7855981 ]



for each image there are 5-7 descriptions, e.g.:

a man with a red helmet on a small moped on a dirt road
man riding a motor bike on a dirt road on the countryside
a man riding on the back of a motorcycle
a dirt path with a young person on a motor bike rests to the foreground of a verdant area with a bridge and a background of cloud wreathed mountains
a man in a red shirt and a red hat is on a motorcycle on a hill side


In [3]:
#split descriptions into tokens
for img_i in range(len(captions)):
    for caption_i in range(len(captions[img_i])):
        sentence = captions[img_i][caption_i] 
        captions[img_i][caption_i] = ["#START#"]+sentence.split(' ')+["#END#"]

In [23]:
voc = []
voc_all = []
for img_i in range(len(captions)):
    for caption_i in range(len(captions[img_i])):
        for el in captions[img_i][caption_i]:
            if el not in voc:
                voc.append(el)
            voc_all.append(el)

In [40]:
# Build a Vocabulary
word_counts = dict()
i = 0
for word in voc:
    word_counts[word] = voc_all.count(word)
    i += 1
    if i % 1000 == 0:
        print i

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000


In [4]:
with open("vocad.txt", 'r') as f:
    voc = f.readlines()
vocab = []
for elem in voc:
    vocab.append(elem[:-1])

In [5]:
with open("dictionary_keys.txt", 'r') as f:
    dic_k = f.readlines()
dict_k = []
for elem in dic_k:
    dict_k.append(elem[:-1])

In [6]:
with open("dictionary_values.txt", 'r') as f:
    dic_v = f.readlines()
dict_v = []
for elem in dic_v:
    dict_v.append(int(elem[:-1]))

In [7]:
word_counts = dict()
for i in range(len(dict_k)):
     word_counts[dict_k[i]] = dict_v[i]

In [8]:
vocab  = ['#UNK#', '#START#', '#END#']
vocab += [k for k, v in word_counts.items() if v >= 5]
n_tokens = len(vocab)

assert 10000 <= n_tokens <= 10500

word_to_index = {w: i for i, w in enumerate(vocab)}

In [9]:
PAD_ix = -1
UNK_ix = vocab.index('#UNK#')

def as_matrix(sequences,max_len=None):
    max_len = max_len or max(map(len,sequences))
    
    matrix = np.zeros((len(sequences),max_len),dtype='int32')+PAD_ix
    for i,seq in enumerate(sequences):
        row_ix = [word_to_index.get(word,UNK_ix) for word in seq[:max_len]]
        matrix[i,:len(row_ix)] = row_ix
    
    return matrix

In [10]:
#try it out on several descriptions of a random image
as_matrix(captions[1338])

array([[ 8508,  8739,  8316,  6620,  1354,  8739,   133,  7732,  2469,
         4928,  3249,  8168,    -1,    -1,    -1,    -1],
       [ 8508,  8739,  6620,  1847,  8739,  9929,  5187,  5750, 10051,
         8739,  3249,  8274, 10133,  4771,  8168,    -1],
       [ 8508,  8739,  6620,  9088,  8739,  7479,  4928,  3249,  8274,
        10133,  4771,  8168,    -1,    -1,    -1,    -1],
       [ 8508,  8739,  6620,  1847, 10133,  5750,  5187,  9929,   132,
         3169,  4928,  3249,  8274, 10133,  4771,  8168],
       [ 8508,  8739,  6620,   328,   723,  6620,  8698,   132,  7513,
         3249,  8274, 10133,  4771,  8168,    -1,    -1]])

In [43]:
keys_str = []
for elem in word_counts.keys():
    keys_str.append(elem.encode('utf8'))

In [95]:
# Сохранила предобработанные данные
np.savetxt('dictionary_values.txt',word_counts.values(), fmt = ('%.18e'), delimiter=',')

In [119]:
np.savetxt('dictionary_keys.txt',keys_str,fmt = '%s', delimiter=',')

In [121]:
np.savetxt('vocad.txt', vocab, fmt= '%s', delimiter=',')

### Mah Neural Networksa

In [11]:
# network shapes. 
CNN_FEATURE_SIZE = img_codes.shape[1]
EMBED_SIZE = 128 #pls change me if u want
LSTM_UNITS = 200 #pls change me if u want

In [12]:
import theano
import lasagne
import theano.tensor as T
from lasagne.layers import *

C:\Users\GTA\Anaconda2\lib\site-packages\theano\tensor\signal\downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [13]:
# Input Variable
sentences = T.imatrix()# [batch_size x time] of word ids
image_vectors = T.matrix() # [batch size x unit] of CNN image features
sentence_mask = T.neq(sentences, PAD_ix)

In [36]:
sentences.shape

Shape.0

In [ ]:
sentence_mask = T.neq(sentences, PAD_ix)
words = InputLayer((None, None), sentences)
masks = InputLayer((None, None), sentence_mask)
words_emb = EmbeddingLayer(words, n_tokens, EMBED_SIZE)
image_fea = InputLayer((None, CNN_FEATURE_SIZE), image_vec)
image_emb = DropoutLayer(image_fea, 0.5)
image_emb = DenseLayer(image_emb, LSTM_UNITS)
decoder = LSTMLayer(
words_emb, num_units=LSTM_UNITS,
cell_init=image_emb, mask_input=mask)
decoder = Reshape (...)
loss = categorical_cross_enrtopy (...)

In [15]:
#network inputs
l_words = InputLayer((None, None), sentences)
l_mask = InputLayer((None, None), sentence_mask)

#embeddings for words 
############# TO CODE IT BY YOURSELF ##################
l_word_embeddings = EmbeddingLayer(l_words, n_tokens, EMBED_SIZE)

In [57]:
# input layer for image features
l_image_features = InputLayer((None, CNN_FEATURE_SIZE), image_vectors)

############# TO CODE IT BY YOURSELF ##################
#convert 1000 image features from googlenet to whatever LSTM_UNITS you have set
#it's also a good idea to add some dropout here and there

l_image_features_small = DropoutLayer(l_image_features, p = 0.5) #<Apply Dropout Layer to regularise your Net>
l_image_features_small = DenseLayer(l_image_features_small,LSTM_UNITS) #<Apply Dense to acive LSTM_UNITS size of representation>
assert l_image_features_small.output_shape == (None, LSTM_UNITS)

In [63]:
l_image_features_small

In [67]:
############# TO CODE IT BY YOURSELF ##################
# Concatinate image features and word embedings in one sequence 
decoder = LSTMLayer(l_word_embeddings, 
                    num_units=LSTM_UNITS, 
                    cell_init=l_image_features_small, 
                    mask_input=l_mask,
                    grad_clipping=1)

RuntimeError: cannot initialize parameters: 'spec' is not a numpy array, a Theano shared variable, or a callable

In [27]:
# Decoding of rnn hiden states
from broadcast import BroadcastLayer,UnbroadcastLayer

#apply whatever comes next to each tick of each example in a batch. Equivalent to 2 reshapes
broadcast_decoder_ticks = BroadcastLayer(decoder, (0, 1))
print "broadcasted decoder shape = ",broadcast_decoder_ticks.output_shape

predicted_probabilities_each_tick = DenseLayer(
    broadcast_decoder_ticks,n_tokens, nonlinearity=lasagne.nonlinearities.softmax)

#un-broadcast back into (batch,tick,probabilities)
predicted_probabilities = UnbroadcastLayer(
    predicted_probabilities_each_tick, broadcast_layer=broadcast_decoder_ticks)

print "output shape = ", predicted_probabilities.output_shape

#remove if you know what you're doing (e.g. 1d convolutions or fixed shape)
assert predicted_probabilities.output_shape == (None, None, 10373)

NameError: name 'decoder' is not defined

In [66]:
next_word_probas = get_output(predicted_probabilities)

reference_answers = sentences[:,1:]
output_mask = sentence_mask[:,1:]

#write symbolic loss function to train NN for
loss = lasagne.objectives.categorical_crossentropy(
    next_word_probas[:, :-1].reshape((-1, n_tokens)),
    reference_answers.reshape((-1,))
).reshape(reference_answers.shape)

############# TO CODE IT BY YOURSELF ##################
loss =  <mean over non-PAD tokens>

NameError: name 'predicted_probabilities' is not defined

In [ ]:
#trainable NN weights
############# TO CODE IT BY YOURSELF ##################
weights = <all dnn weigts>
updates = <your favorite optimizer>

In [ ]:
#compile a function that takes input sentence and image mask, outputs loss and updates weights
#please not that your functions must accept image features as FIRST param and sentences as second one
############# TO CODE IT BY YOURSELF ##################
train_step = <>
val_step   = <>

# Training

* You first have to implement a batch generator
* Than the network will get trained the usual way

In [24]:
captions = np.array(captions)

In [25]:
from random import choice

def generate_batch(images,captions,batch_size,max_caption_len=None):
    #sample random numbers for image/caption indicies
    random_image_ix = np.random.randint(0, len(images), size=batch_size)
    
    #get images
    batch_images = images[random_image_ix]
    
    #5-7 captions for each image
    captions_for_batch_images = captions[random_image_ix]
    
    #pick 1 from 5-7 captions for each image
    batch_captions = map(choice, captions_for_batch_images)
    
    #convert to matrix
    batch_captions_ix = as_matrix(batch_captions,max_len=max_caption_len)
    
    return batch_images, batch_captions_ix

In [26]:
generate_batch(img_codes,captions, 3)

(array([[ 0.60496843, -0.2175829 ,  2.45054078, ..., -3.86057615,
          3.22173691, -1.05588841],
        [-3.09767723, -1.64312673, -0.65251952, ..., -1.1166991 ,
          2.94198656, -1.68168533],
        [-2.60519528, -4.79385853, -0.87879199, ..., -4.22373724,
          2.63850451,  3.32209778]], dtype=float32),
 array([[ 8508,  4983,  7179,  3511,  4558,  1644,  3396,  2108, 10336,
          8168,    -1,    -1,    -1],
        [ 8508,  8739,  3590,  7392,  5520,  3080,  1920,  5650,  2730,
          3824,  8168,    -1,    -1],
        [ 8508,   383,  1445,  4046,   206,  2366,  8996,  9833,  8274,
          3396,   773, 10010,  8168]]))

### Main loop
* We recommend you to periodically evaluate the network using the next "apply trained model" block
 *  its safe to interrupt training, run a few examples and start training again

In [27]:
batch_size = 50 #adjust me
n_epochs   = 100 #adjust me
n_batches_per_epoch = 50 #adjust me
n_validation_batches = 5 #how many batches are used for validation after each epoch

In [28]:
from tqdm import tqdm

for epoch in range(n_epochs):
    train_loss=0
    for _ in tqdm(range(n_batches_per_epoch)):
        train_loss += train_step(*generate_batch(img_codes,captions,batch_size))
    train_loss /= n_batches_per_epoch
    
    val_loss=0
    for _ in range(n_validation_batches):
        val_loss += val_step(*generate_batch(img_codes,captions,batch_size))
    val_loss /= n_validation_batches
    
    print('\nEpoch: {}, train loss: {}, val loss: {}'.format(epoch, train_loss, val_loss))

print("Finish :)")

  0%|                                                   | 0/50 [00:00<?, ?it/s]


NameError: name 'train_step' is not defined

### apply trained model

In [ ]:
#the same kind you did last week, but a bit smaller
from pretrained_lenet import build_model,preprocess,MEAN_VALUES

# build googlenet
lenet = build_model()

#load weights
lenet_weights = pickle.load(open('data/blvc_googlenet.pkl'))['param values']
set_all_param_values(lenet["prob"], lenet_weights)

#compile get_features
cnn_input_var = lenet['input'].input_var
cnn_feature_layer = lenet['loss3/classifier']
get_cnn_features = theano.function([cnn_input_var], lasagne.layers.get_output(cnn_feature_layer))

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

#sample image
img = plt.imread('data/Dog-and-Cat.jpg')
img = preprocess(img)

In [ ]:
#deprocess and show, one line :)
from pretrained_lenet import MEAN_VALUES
plt.imshow(np.transpose((img[0] + MEAN_VALUES)[::-1],[1,2,0]).astype('uint8'))

## Generate caption

In [ ]:
last_word_probas_det = get_output(predicted_probabilities,deterministic=False)[:,-1]

get_probs = theano.function([image_vectors,sentences], last_word_probas_det)

#this is exactly the generation function from week5 classwork,
#except now we condition on image features instead of words
def generate_caption(image,caption_prefix = ("START",),t=1,sample=True,max_len=100):
    image_features = get_cnn_features(image)
    caption = list(caption_prefix)
    for _ in range(max_len):
        
        next_word_probs = get_probs(image_features,as_matrix([caption]) ).ravel()
        #apply temperature
        next_word_probs = next_word_probs**t / np.sum(next_word_probs**t)

        if sample:
            next_word = np.random.choice(vocab,p=next_word_probs) 
        else:
            next_word = vocab[np.argmax(next_word_probs)]

        caption.append(next_word)

        if next_word=="#END#":
            break
            
    return caption

In [ ]:
for i in range(10):
    print ' '.join(generate_caption(img,t=1.)[1:-1])

# Bonus Part
- Use ResNet Instead of GoogLeNet
- Use W2V as embedding
- Use Attention :) 

# Pass Assignment https://goo.gl/forms/2qqVtfepn0t1aDgh1 